Retreive 20 records including the last 4 columns from the GCS's file generated from ex01.ipynb
Using "gemini-2.5-flash", ask it to summarize the reports.

- Pre-requisite : Generate API keys through [Google AI Studio](https://aistudio.google.com/app/apikey)

In [1]:
import json
import os

from dotenv import load_dotenv
from google import genai
from google.oauth2 import service_account
from google.cloud import storage

In [2]:
load_dotenv(dotenv_path='../.env')

True

In [3]:
genai_api_key = os.getenv("GCP_GENAI_API_KEY")
service_account_key = os.getenv("GCP_SERVICE_ACCOUNT_KEY")
project_id = os.getenv("GCP_PROJECT_ID")
bucket_name = os.getenv("GCP_BUCKET_NAME")
file_name = f"sf_police_report/2025-09-01.json"

In [4]:
def retrieve_data_from_gcs(service_account_key: str,
                           project_id: str,
                           bucket_name: str,
                           file_name: str,
                           key_list: list
                           ) -> list:
    credentials = service_account.Credentials.from_service_account_file(service_account_key)
    client = storage.Client(project=project_id,
                            credentials=credentials)
    bucket = client.bucket(bucket_name)
    file = bucket.blob(file_name)
    content = json.loads(file.download_as_string())

    output = []
    for data in content:
        row = []
        
        for key in key_list:
            row.append(data.get(key, None))
        output.append(row)
    return output

In [5]:
key_list = ['incident_datetime', 'report_datetime', 'incident_code',
            'incident_category', 'incident_description', 'latitude',
            'longitude', 'police_district']
data = retrieve_data_from_gcs(service_account_key, 
                              project_id,
                              bucket_name,
                              file_name,
                              key_list)

NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/msds692-class-example-bjm/o/sf_police_report%2F2025-09-01.json?alt=media: No such object: msds692-class-example-bjm/sf_police_report/2025-09-01.json: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

In [ ]:
filtered_data = [row[-4:] for row in data][:20]

In [ ]:
client = genai.Client()

In [ ]:
model_name = "gemini-2.5-flash"

In [ ]:
prompt_content = f"There has been a police report\
on the following list of [description, lon, lat, district] : {filtered_data} recently.\
Summarize the reports"

In [ ]:
response = client.models.generate_content(
    model=model_name,
    contents=prompt_content
)

In [ ]:
response.text

'A total of **20 police reports** were recently filed, covering a diverse range of incidents across several districts.\n\n**Key Trends and Categories:**\n\n1.  **Vehicle-Related Incidents (7 reports):** This is the most common category, including:\n    *   **Theft from Vehicles:** 3 instances of "Theft, From Locked Vehicle, >$950" (Northern, Southern, Park) and 1 instance of "Theft, From Unlocked Vehicle, >$950" (Southern).\n    *   1 "Vehicle, Stolen, Other Vehicle" (Park).\n    *   1 "Vehicle, Recovered, Auto" (Out of SF).\n    *   1 "Arson of Vehicle" (Tenderloin).\n\n2.  **Property Crimes (Excluding Vehicle Theft) (3 reports):**\n    *   2 "Lost Property" reports (Park, Tenderloin).\n    *   1 "Malicious Mischief, Vandalism to Property" (Taraval).\n\n3.  **Assault, Disorder, and Other Interpersonal Incidents (6 reports):**\n    *   1 "Battery" (Park).\n    *   1 "False Personation" (Park).\n    *   1 "Resisting, Delaying, or Obstructing Peace Officer Duties" (Mission).\n    *   1 "